In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc

In [ ]:
def get_atoms(text, fontsize):
    myfont = ImageFont.truetype("fonts/ipaexg.ttf", fontsize)
    img = Image.new('1', (200, 200), 'white')
    draw = ImageDraw.Draw(img)
    draw.text((10, 10), text, font=myfont)

    w, h = img.size
    qx = []
    qy = []
    for x in range(w):
        for y in range(h):
            v = img.getpixel((x, y))
            if v is 0:
                qx.append(float(x))
                qy.append(h - float(y))
    nx = np.array(qx)
    ny = np.array(qy)
    nx -= np.min(nx)
    ny -= np.min(ny)
    return nx, ny

In [ ]:
def get_bonds(qx, qy):
    n = len(qx)
    bonds = []
    for i in range(n-1):
        (xi, yi) = qx[i], qy[i]
        for j in range(i + 1, n):
            (xj, yj) = qx[j], qy[j]
            r2 = (xi - xj) ** 2 + (yi - yj) ** 2
            if r2 < 3.1:
                bonds.append((i, j, r2))
    return bonds

In [ ]:
def calculate(vx, vy, qx, qy, bonds):
    n = len(vx)
    dt = 0.01
    G = 0.2
    K = 1000.0

    qx += vx * dt
    qy += vy * dt

    for i, j, l in bonds:
        dx = qx[j] - qx[i]
        dy = qy[j] - qy[i]
        r2 = dx ** 2 + dy ** 2
        f = K * (r2 - l)
        vx[i] += f*dx*dt
        vy[i] += f*dy*dt
        vx[j] -= f*dx*dt
        vy[j] -= f*dy*dt

    vy -= G * dt

    for i in range(n):
        if qy[i] < 0.0:
            vy[i] -= 10.0 * qy[i] * dt

In [ ]:
def simulate(qx, qy, bonds):
    vx = np.zeros_like(qx)
    vy = np.zeros_like(qx)
    w = np.max(qx).astype(np.int)
    h = np.max(qy).astype(np.int)
    imgs = []
    for i in range(2000):
        if i % 100 is 0:
            img = get_img(qx, qy, w, h)
            imgs.append(img.copy())
            plt.imshow(img)
            plt.show()
        calculate(vx, vy, qx, qy, bonds)
    return imgs

In [ ]:
def get_img(qx, qy, w, h):
    img = np.zeros((h,w))
    for x,y in zip(qx,qy):
        x = int(x)
        y = int(y)
        if x in range(w) and y in range(h):
            img[h-y-1][x] = 1.0
    return img


In [ ]:
qx, qy = get_atoms("スパコン",32)
bonds = get_bonds(qx, qy)
%time imgs = simulate(qx, qy, bonds)


In [ ]:
fig = plt.figure()
im = plt.imshow(imgs[0],cmap="plasma")

In [ ]:
def update(i):
    im.set_array(imgs[i])

ani = animation.FuncAnimation(fig, update,frames=len(imgs))
rc('animation', html='jshtml')
ani